# Partie 3.1 - Statistiques descriptives

Ce notebook realise une analyse statistique descriptive complete des donnees de consommation energetique
des batiments publics. Il couvre les statistiques par type d'energie, par type de batiment, par commune,
par classe DPE, ainsi que l'evolution temporelle et la comparaison theorique vs reelle.

**Donnees en entree :** `../output/consommations_enrichies.parquet`  
**Periode :** 2023-2024  
**Perimetre :** 146 batiments, 15 communes, 5 types de batiments, 3 types d'energie

In [1]:
# ============================================================
# Imports et chargement des donnees enrichies
# ============================================================
import pandas as pd
import numpy as np
import os
import warnings

warnings.filterwarnings('ignore')

# Options d'affichage pour des tableaux lisibles
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:,.2f}'.format)
pd.set_option('display.width', 200)

# Chargement du fichier parquet enrichi
chemin_parquet = os.path.join('..', 'output', 'consommations_enrichies.parquet')
df = pd.read_parquet(chemin_parquet)

print(f"Nombre de lignes : {len(df):,}")
print(f"Nombre de colonnes : {df.shape[1]}")
print(f"Periode : {df['timestamp'].min()} -> {df['timestamp'].max()}")
print(f"Nombre de batiments : {df['batiment_id'].nunique()}")
print(f"Types d'energie : {sorted(df['type_energie'].unique())}")
print(f"Types de batiment : {sorted(df['type'].unique())}")
print(f"Communes : {df['commune'].nunique()}")
print()
print("Apercu des donnees :")
df.head(3)

FileNotFoundError: [Errno 2] No such file or directory: '../output/consommations_enrichies.parquet'

## 1. Statistiques par type d'energie

In [ ]:
# ============================================================
# Statistiques descriptives detaillees par type d'energie
# ============================================================

def statistiques_par_energie(df):
    """Calcule les statistiques descriptives de consommation pour chaque type d'energie."""
    resultats = []
    
    for energie in sorted(df['type_energie'].unique()):
        sous_df = df[df['type_energie'] == energie]['consommation']
        
        stats = {
            'Type d\'energie': energie,
            'Nombre de mesures': int(sous_df.count()),
            'Moyenne': sous_df.mean(),
            'Ecart-type': sous_df.std(),
            'Minimum': sous_df.min(),
            'Q1 (25%)': sous_df.quantile(0.25),
            'Mediane (50%)': sous_df.median(),
            'Q3 (75%)': sous_df.quantile(0.75),
            'Maximum': sous_df.max(),
            'IQR': sous_df.quantile(0.75) - sous_df.quantile(0.25),
            'Coeff. de variation (%)': (sous_df.std() / sous_df.mean()) * 100 if sous_df.mean() != 0 else np.nan
        }
        resultats.append(stats)
    
    return pd.DataFrame(resultats)

# Calcul et affichage
stats_energie = statistiques_par_energie(df)
print("=" * 80)
print("STATISTIQUES DESCRIPTIVES PAR TYPE D'ENERGIE")
print("=" * 80)
print()

# Affichage avec mise en forme
affichage = stats_energie.set_index("Type d'energie")
display(affichage.style.format({
    'Nombre de mesures': '{:,.0f}',
    'Moyenne': '{:,.2f}',
    'Ecart-type': '{:,.2f}',
    'Minimum': '{:,.2f}',
    'Q1 (25%)': '{:,.2f}',
    'Mediane (50%)': '{:,.2f}',
    'Q3 (75%)': '{:,.2f}',
    'Maximum': '{:,.2f}',
    'IQR': '{:,.2f}',
    'Coeff. de variation (%)': '{:,.1f}%'
}).set_caption("Statistiques de consommation par type d'energie"))

print()
print("Interpretation :")
for _, row in stats_energie.iterrows():
    print(f"  - {row['Type d\'energie']} : moyenne = {row['Moyenne']:,.2f}, "
          f"mediane = {row['Mediane (50%)']:,.2f}, "
          f"CV = {row['Coeff. de variation (%)']:.1f}%")

## 2. Statistiques par type de batiment

In [ ]:
# ============================================================
# Statistiques agregees par type de batiment
# ============================================================

# Nombre de batiments par type
nb_batiments_par_type = df.groupby('type')['batiment_id'].nunique().reset_index()
nb_batiments_par_type.columns = ['Type de batiment', 'Nombre de batiments']

# Consommation totale moyenne par batiment et par type
conso_totale_par_batiment = df.groupby(['type', 'batiment_id']).agg(
    consommation_totale=('consommation', 'sum'),
    cout_total=('cout', 'sum'),
    surface=('surface_m2', 'first')
).reset_index()

# Calcul de la consommation par m2 pour chaque batiment
conso_totale_par_batiment['conso_par_m2'] = (
    conso_totale_par_batiment['consommation_totale'] / conso_totale_par_batiment['surface']
)

# Agregation par type de batiment
stats_type_batiment = conso_totale_par_batiment.groupby('type').agg(
    consommation_totale_moyenne=('consommation_totale', 'mean'),
    cout_moyen=('cout_total', 'mean'),
    conso_par_m2_moyenne=('conso_par_m2', 'mean'),
    surface_moyenne=('surface', 'mean')
).reset_index()

# Fusion avec le nombre de batiments
stats_type_batiment = stats_type_batiment.merge(
    nb_batiments_par_type, left_on='type', right_on='Type de batiment'
).drop(columns=['Type de batiment'])

# Renommage des colonnes pour l'affichage
stats_type_batiment.columns = [
    'Type de batiment', 
    'Consommation totale moyenne',
    'Cout moyen (EUR)', 
    'Consommation moyenne par m2',
    'Surface moyenne (m2)',
    'Nombre de batiments'
]

# Tri par consommation totale moyenne decroissante
stats_type_batiment = stats_type_batiment.sort_values(
    'Consommation totale moyenne', ascending=False
).reset_index(drop=True)

print("=" * 80)
print("STATISTIQUES PAR TYPE DE BATIMENT")
print("=" * 80)
print()

display(stats_type_batiment.style.format({
    'Consommation totale moyenne': '{:,.2f}',
    'Cout moyen (EUR)': '{:,.2f}',
    'Consommation moyenne par m2': '{:,.2f}',
    'Surface moyenne (m2)': '{:,.0f}',
    'Nombre de batiments': '{:d}'
}).set_caption("Statistiques de consommation par type de batiment").hide(axis='index'))

print()
print("Observations :")
type_max = stats_type_batiment.iloc[0]
type_min = stats_type_batiment.iloc[-1]
print(f"  - Type le plus energivore : {type_max['Type de batiment']} "
      f"({type_max['Consommation moyenne par m2']:,.2f} par m2)")
print(f"  - Type le moins energivore : {type_min['Type de batiment']} "
      f"({type_min['Consommation moyenne par m2']:,.2f} par m2)")

## 3. Statistiques par commune

In [ ]:
# ============================================================
# Statistiques agregees par commune
# ============================================================

# Consommation totale par commune et par type d'energie (pivot)
conso_par_commune_energie = df.groupby(['commune', 'type_energie'])['consommation'].sum().unstack(
    fill_value=0
)

# Cout total par commune
cout_par_commune = df.groupby('commune')['cout'].sum()

# Nombre de batiments par commune
nb_batiments_par_commune = df.groupby('commune')['batiment_id'].nunique()

# Consommation totale toutes energies
conso_totale_commune = df.groupby('commune')['consommation'].sum()

# Assemblage du tableau de synthese
stats_commune = pd.DataFrame({
    'Nombre de batiments': nb_batiments_par_commune,
    'Cout total (EUR)': cout_par_commune,
    'Consommation totale': conso_totale_commune
})

# Ajout des colonnes par type d'energie
for col in conso_par_commune_energie.columns:
    stats_commune[f'Conso. {col}'] = conso_par_commune_energie[col]

# Tri par cout total decroissant
stats_commune = stats_commune.sort_values('Cout total (EUR)', ascending=False)

print("=" * 80)
print("STATISTIQUES PAR COMMUNE")
print("=" * 80)
print()

# Formatage pour l'affichage
format_dict = {
    'Nombre de batiments': '{:d}',
    'Cout total (EUR)': '{:,.2f}',
    'Consommation totale': '{:,.2f}'
}
for col in stats_commune.columns:
    if col.startswith('Conso.'):
        format_dict[col] = '{:,.2f}'

display(stats_commune.style.format(format_dict).set_caption(
    "Consommation et couts par commune (trie par cout total decroissant)"
))

print()
print(f"Commune la plus couteuse : {stats_commune.index[0]} "
      f"({stats_commune.iloc[0]['Cout total (EUR)']:,.2f} EUR)")
print(f"Commune la moins couteuse : {stats_commune.index[-1]} "
      f"({stats_commune.iloc[-1]['Cout total (EUR)']:,.2f} EUR)")
print(f"Cout moyen par commune : {stats_commune['Cout total (EUR)'].mean():,.2f} EUR")

## 4. Batiments les plus et moins energivores

In [ ]:
# ============================================================
# Top 10 et Bottom 10 des batiments par consommation par m2
# ============================================================

# Calcul de la consommation totale par m2 pour chaque batiment
conso_batiment = df.groupby('batiment_id').agg(
    nom=('nom', 'first'),
    type_batiment=('type', 'first'),
    commune=('commune', 'first'),
    classe_dpe=('classe_energetique', 'first'),
    surface_m2=('surface_m2', 'first'),
    annee_construction=('annee_construction', 'first'),
    consommation_totale=('consommation', 'sum'),
    cout_total=('cout', 'sum')
).reset_index()

# Consommation par m2
conso_batiment['conso_par_m2'] = (
    conso_batiment['consommation_totale'] / conso_batiment['surface_m2']
)

# Tri par consommation par m2
conso_batiment_tri = conso_batiment.sort_values('conso_par_m2', ascending=False)

# Colonnes a afficher
colonnes_affichage = [
    'nom', 'type_batiment', 'commune', 'classe_dpe', 
    'surface_m2', 'annee_construction', 'consommation_totale', 'conso_par_m2', 'cout_total'
]
noms_colonnes = [
    'Nom', 'Type', 'Commune', 'Classe DPE',
    'Surface (m2)', 'Annee construction', 'Conso. totale', 'Conso. par m2', 'Cout total (EUR)'
]

# --- TOP 10 : Les plus energivores ---
top10 = conso_batiment_tri.head(10)[colonnes_affichage].reset_index(drop=True)
top10.columns = noms_colonnes
top10.index = range(1, 11)
top10.index.name = 'Rang'

print("=" * 80)
print("TOP 10 - BATIMENTS LES PLUS ENERGIVORES (par consommation/m2)")
print("=" * 80)
print()
display(top10.style.format({
    'Surface (m2)': '{:,.0f}',
    'Annee construction': '{:.0f}',
    'Conso. totale': '{:,.2f}',
    'Conso. par m2': '{:,.2f}',
    'Cout total (EUR)': '{:,.2f}'
}).set_caption("Les 10 batiments les plus energivores"))

print()

# --- BOTTOM 10 : Les moins energivores ---
bottom10 = conso_batiment_tri.tail(10)[colonnes_affichage].reset_index(drop=True)
bottom10.columns = noms_colonnes
bottom10.index = range(1, 11)
bottom10.index.name = 'Rang'

print("=" * 80)
print("BOTTOM 10 - BATIMENTS LES MOINS ENERGIVORES (par consommation/m2)")
print("=" * 80)
print()
display(bottom10.style.format({
    'Surface (m2)': '{:,.0f}',
    'Annee construction': '{:.0f}',
    'Conso. totale': '{:,.2f}',
    'Conso. par m2': '{:,.2f}',
    'Cout total (EUR)': '{:,.2f}'
}).set_caption("Les 10 batiments les moins energivores"))

print()
print("Analyse :")
print(f"  - Ecart entre le plus et le moins energivore : "
      f"{conso_batiment_tri.iloc[0]['conso_par_m2']:,.2f} vs "
      f"{conso_batiment_tri.iloc[-1]['conso_par_m2']:,.2f} (par m2)")
print(f"  - Ratio max/min : {conso_batiment_tri.iloc[0]['conso_par_m2'] / conso_batiment_tri.iloc[-1]['conso_par_m2']:.1f}x")

## 5. Repartition des consommations par classe energetique DPE

In [ ]:
# ============================================================
# Analyse par classe energetique DPE
# ============================================================

# Statistiques par batiment d'abord, puis agregation par classe DPE
stats_batiment_dpe = df.groupby(['classe_energetique', 'batiment_id']).agg(
    consommation_totale=('consommation', 'sum'),
    cout_total=('cout', 'sum'),
    surface_m2=('surface_m2', 'first')
).reset_index()

stats_batiment_dpe['conso_par_m2'] = (
    stats_batiment_dpe['consommation_totale'] / stats_batiment_dpe['surface_m2']
)

# Agregation par classe DPE
stats_dpe = stats_batiment_dpe.groupby('classe_energetique').agg(
    nombre_batiments=('batiment_id', 'nunique'),
    consommation_moyenne=('consommation_totale', 'mean'),
    cout_moyen=('cout_total', 'mean'),
    conso_par_m2_moyenne=('conso_par_m2', 'mean'),
    conso_par_m2_mediane=('conso_par_m2', 'median'),
    conso_par_m2_std=('conso_par_m2', 'std')
).reset_index()

# Tri par classe DPE (A -> G)
ordre_dpe = ['A', 'B', 'C', 'D', 'E', 'F', 'G']
stats_dpe['classe_energetique'] = pd.Categorical(
    stats_dpe['classe_energetique'], categories=ordre_dpe, ordered=True
)
stats_dpe = stats_dpe.sort_values('classe_energetique').reset_index(drop=True)

# Renommage pour affichage
stats_dpe_affichage = stats_dpe.rename(columns={
    'classe_energetique': 'Classe DPE',
    'nombre_batiments': 'Nb. batiments',
    'consommation_moyenne': 'Conso. moyenne totale',
    'cout_moyen': 'Cout moyen (EUR)',
    'conso_par_m2_moyenne': 'Conso. moy. par m2',
    'conso_par_m2_mediane': 'Conso. med. par m2',
    'conso_par_m2_std': 'Ecart-type conso. par m2'
}).set_index('Classe DPE')

print("=" * 80)
print("REPARTITION DES CONSOMMATIONS PAR CLASSE ENERGETIQUE DPE")
print("=" * 80)
print()

display(stats_dpe_affichage.style.format({
    'Nb. batiments': '{:d}',
    'Conso. moyenne totale': '{:,.2f}',
    'Cout moyen (EUR)': '{:,.2f}',
    'Conso. moy. par m2': '{:,.2f}',
    'Conso. med. par m2': '{:,.2f}',
    'Ecart-type conso. par m2': '{:,.2f}'
}).set_caption("Statistiques par classe DPE (A = meilleure, G = pire)"))

# Verification de la correlation DPE vs consommation reelle
print()
print("Verification de la correlation DPE / consommation reelle :")
print("-" * 60)

conso_par_classe = stats_dpe.set_index('classe_energetique')['conso_par_m2_moyenne']
est_croissant = all(
    conso_par_classe.iloc[i] <= conso_par_classe.iloc[i + 1]
    for i in range(len(conso_par_classe) - 1)
)

if est_croissant:
    print("  -> La consommation reelle est strictement croissante de A vers G.")
    print("  -> La classification DPE est coherente avec les consommations mesurees.")
else:
    print("  -> La consommation reelle n'est PAS strictement croissante de A vers G.")
    print("  -> Il existe des ecarts entre la classification DPE et les consommations mesurees.")
    print()
    print("  Detail par classe :")
    for classe in ordre_dpe:
        if classe in conso_par_classe.index:
            print(f"    Classe {classe} : {conso_par_classe[classe]:,.2f} par m2")

## 6. Evolution temporelle

In [ ]:
# ============================================================
# Analyse de l'evolution temporelle des consommations
# ============================================================

# --- 6.1 Consommation mensuelle par type d'energie ---
print("=" * 80)
print("EVOLUTION TEMPORELLE DES CONSOMMATIONS")
print("=" * 80)
print()

# Extraction annee et mois
df['annee'] = df['timestamp'].dt.year
df['annee_mois'] = df['timestamp'].dt.to_period('M')

# Consommation mensuelle par type d'energie
conso_mensuelle = df.groupby(['annee_mois', 'type_energie'])['consommation'].sum().unstack(fill_value=0)
conso_mensuelle.index = conso_mensuelle.index.astype(str)

print("--- Consommation mensuelle par type d'energie ---")
print()

# Formatage pour affichage
format_dict_mensuel = {col: '{:,.2f}' for col in conso_mensuelle.columns}
display(conso_mensuelle.style.format(format_dict_mensuel).set_caption(
    "Consommation mensuelle totale par type d'energie"
))

# --- 6.2 Comparaison annuelle 2023 vs 2024 ---
print()
print("--- Comparaison annuelle 2023 vs 2024 ---")
print()

conso_annuelle = df.groupby(['annee', 'type_energie'])['consommation'].sum().unstack(fill_value=0)

if 2023 in conso_annuelle.index and 2024 in conso_annuelle.index:
    comparaison = pd.DataFrame({
        '2023': conso_annuelle.loc[2023],
        '2024': conso_annuelle.loc[2024]
    })
    comparaison['Evolution (abs.)'] = comparaison['2024'] - comparaison['2023']
    comparaison['Evolution (%)'] = (
        (comparaison['2024'] - comparaison['2023']) / comparaison['2023'] * 100
    )
    
    display(comparaison.style.format({
        '2023': '{:,.2f}',
        '2024': '{:,.2f}',
        'Evolution (abs.)': '{:+,.2f}',
        'Evolution (%)': '{:+.2f}%'
    }).set_caption("Evolution annuelle par type d'energie (2023 vs 2024)"))
    
    # Total toutes energies
    total_2023 = conso_annuelle.loc[2023].sum()
    total_2024 = conso_annuelle.loc[2024].sum()
    evolution_totale = (total_2024 - total_2023) / total_2023 * 100
    print(f"\n  Evolution totale toutes energies : {evolution_totale:+.2f}%")
else:
    print("  Donnees insuffisantes pour la comparaison annuelle.")

# --- 6.3 Analyse de la saisonnalite ---
print()
print("--- Analyse de la saisonnalite ---")
print()

# Consommation moyenne par saison et type d'energie
ordre_saisons = ['hiver', 'printemps', 'ete', 'automne']
conso_saison = df.groupby(['saison', 'type_energie'])['consommation'].mean().unstack(fill_value=0)

# Reordonner les saisons si les valeurs existent
saisons_presentes = [s for s in ordre_saisons if s in conso_saison.index]
conso_saison = conso_saison.reindex(saisons_presentes)

# Ajout d'une colonne total
conso_saison['Total moyen'] = conso_saison.sum(axis=1)

format_dict_saison = {col: '{:,.2f}' for col in conso_saison.columns}
display(conso_saison.style.format(format_dict_saison).set_caption(
    "Consommation moyenne par saison et type d'energie"
))

print()
# Identification de la saison la plus consommatrice par energie
print("Saison la plus consommatrice par type d'energie :")
for col in conso_saison.columns[:-1]:  # Exclure 'Total moyen'
    saison_max = conso_saison[col].idxmax()
    valeur_max = conso_saison[col].max()
    print(f"  - {col} : {saison_max} ({valeur_max:,.2f})")

## 7. Comparaison consommation theorique vs reelle

In [ ]:
# ============================================================
# Comparaison entre consommation theorique DPE et reelle
# ============================================================

# Plages de consommation theorique par classe DPE (en kWh/m2/an)
# Basees sur la reglementation DPE francaise
consommation_theorique_dpe = {
    'A': {'min': 0, 'max': 70, 'moyenne': 35},
    'B': {'min': 70, 'max': 110, 'moyenne': 90},
    'C': {'min': 110, 'max': 180, 'moyenne': 145},
    'D': {'min': 180, 'max': 250, 'moyenne': 215},
    'E': {'min': 250, 'max': 330, 'moyenne': 290},
    'F': {'min': 330, 'max': 420, 'moyenne': 375},
    'G': {'min': 420, 'max': 600, 'moyenne': 510}
}

# Calcul de la consommation reelle annualisee par m2 pour chaque batiment
# On utilise uniquement electricite et gaz pour la comparaison DPE (pas l'eau)
df_energie = df[df['type_energie'].isin(['electricite', 'gaz'])].copy()

# Nombre d'annees de donnees par batiment
nb_annees = df_energie.groupby('batiment_id')['annee'].nunique()

conso_reelle_batiment = df_energie.groupby('batiment_id').agg(
    classe_dpe=('classe_energetique', 'first'),
    surface_m2=('surface_m2', 'first'),
    consommation_totale=('consommation', 'sum')
).reset_index()

conso_reelle_batiment = conso_reelle_batiment.merge(
    nb_annees.rename('nb_annees'), on='batiment_id'
)

# Consommation annuelle par m2 (normalisee par le nombre d'annees)
conso_reelle_batiment['conso_reelle_annuelle_m2'] = (
    conso_reelle_batiment['consommation_totale'] 
    / conso_reelle_batiment['surface_m2'] 
    / conso_reelle_batiment['nb_annees']
)

# Ajout de la consommation theorique
conso_reelle_batiment['conso_theorique_moy'] = conso_reelle_batiment['classe_dpe'].map(
    {k: v['moyenne'] for k, v in consommation_theorique_dpe.items()}
)
conso_reelle_batiment['conso_theorique_min'] = conso_reelle_batiment['classe_dpe'].map(
    {k: v['min'] for k, v in consommation_theorique_dpe.items()}
)
conso_reelle_batiment['conso_theorique_max'] = conso_reelle_batiment['classe_dpe'].map(
    {k: v['max'] for k, v in consommation_theorique_dpe.items()}
)

# Ecart entre reel et theorique
conso_reelle_batiment['ecart_absolu'] = (
    conso_reelle_batiment['conso_reelle_annuelle_m2'] 
    - conso_reelle_batiment['conso_theorique_moy']
)
conso_reelle_batiment['ecart_pct'] = (
    conso_reelle_batiment['ecart_absolu'] 
    / conso_reelle_batiment['conso_theorique_moy'] * 100
)

# Le batiment est-il dans la plage theorique ?
conso_reelle_batiment['dans_plage_dpe'] = (
    (conso_reelle_batiment['conso_reelle_annuelle_m2'] >= conso_reelle_batiment['conso_theorique_min'])
    & (conso_reelle_batiment['conso_reelle_annuelle_m2'] <= conso_reelle_batiment['conso_theorique_max'])
)

# Synthese par classe DPE
comparaison_dpe = conso_reelle_batiment.groupby('classe_dpe').agg(
    nb_batiments=('batiment_id', 'count'),
    conso_reelle_moy=('conso_reelle_annuelle_m2', 'mean'),
    conso_theorique_moy=('conso_theorique_moy', 'first'),
    conso_theorique_min=('conso_theorique_min', 'first'),
    conso_theorique_max=('conso_theorique_max', 'first'),
    ecart_moyen_pct=('ecart_pct', 'mean'),
    pct_dans_plage=('dans_plage_dpe', 'mean')
).reset_index()

# Tri par classe
comparaison_dpe['classe_dpe'] = pd.Categorical(
    comparaison_dpe['classe_dpe'], categories=ordre_dpe, ordered=True
)
comparaison_dpe = comparaison_dpe.sort_values('classe_dpe').reset_index(drop=True)
comparaison_dpe['pct_dans_plage'] = comparaison_dpe['pct_dans_plage'] * 100

# Renommage pour affichage
comparaison_dpe_affichage = comparaison_dpe.rename(columns={
    'classe_dpe': 'Classe DPE',
    'nb_batiments': 'Nb. batiments',
    'conso_reelle_moy': 'Conso. reelle moy. (kWh/m2/an)',
    'conso_theorique_moy': 'Conso. theorique moy. (kWh/m2/an)',
    'conso_theorique_min': 'Plage theorique min',
    'conso_theorique_max': 'Plage theorique max',
    'ecart_moyen_pct': 'Ecart moyen (%)',
    'pct_dans_plage': 'Batiments dans plage DPE (%)'
}).set_index('Classe DPE')

print("=" * 80)
print("COMPARAISON CONSOMMATION THEORIQUE DPE vs REELLE")
print("(basee sur electricite + gaz uniquement, normalisee par m2 et par an)")
print("=" * 80)
print()

display(comparaison_dpe_affichage.style.format({
    'Nb. batiments': '{:d}',
    'Conso. reelle moy. (kWh/m2/an)': '{:,.2f}',
    'Conso. theorique moy. (kWh/m2/an)': '{:,.2f}',
    'Plage theorique min': '{:,.0f}',
    'Plage theorique max': '{:,.0f}',
    'Ecart moyen (%)': '{:+.1f}%',
    'Batiments dans plage DPE (%)': '{:.1f}%'
}).set_caption("Comparaison theorique vs reelle par classe DPE"))

print()
nb_dans_plage = conso_reelle_batiment['dans_plage_dpe'].sum()
nb_total = len(conso_reelle_batiment)
print(f"Batiments dans la plage theorique DPE : {nb_dans_plage}/{nb_total} "
      f"({nb_dans_plage/nb_total*100:.1f}%)")
print(f"Batiments hors plage theorique DPE : {nb_total - nb_dans_plage}/{nb_total} "
      f"({(nb_total - nb_dans_plage)/nb_total*100:.1f}%)")

# Identification des plus gros ecarts
print()
print("Ecart moyen par classe DPE :")
for _, row in comparaison_dpe.iterrows():
    signe = "+" if row['ecart_moyen_pct'] > 0 else ""
    direction = "surconsommation" if row['ecart_moyen_pct'] > 0 else "sous-consommation"
    print(f"  Classe {row['classe_dpe']} : {signe}{row['ecart_moyen_pct']:.1f}% ({direction})")

## 8. Export des tableaux de synthese

In [ ]:
# ============================================================
# Export des tableaux de synthese en CSV
# ============================================================

chemin_sortie = os.path.join('..', 'output')
os.makedirs(chemin_sortie, exist_ok=True)

exports = {}

# 1. Statistiques par type d'energie
fichier_1 = os.path.join(chemin_sortie, 'stats_par_type_energie.csv')
stats_energie.to_csv(fichier_1, index=False, sep=';', encoding='utf-8-sig')
exports['Statistiques par type d\'energie'] = fichier_1

# 2. Statistiques par type de batiment
fichier_2 = os.path.join(chemin_sortie, 'stats_par_type_batiment.csv')
stats_type_batiment.to_csv(fichier_2, index=False, sep=';', encoding='utf-8-sig')
exports['Statistiques par type de batiment'] = fichier_2

# 3. Statistiques par commune
fichier_3 = os.path.join(chemin_sortie, 'stats_par_commune.csv')
stats_commune.to_csv(fichier_3, sep=';', encoding='utf-8-sig')
exports['Statistiques par commune'] = fichier_3

# 4. Classement des batiments par consommation par m2
fichier_4 = os.path.join(chemin_sortie, 'classement_batiments_conso_m2.csv')
conso_batiment_tri[colonnes_affichage].to_csv(fichier_4, index=False, sep=';', encoding='utf-8-sig')
exports['Classement batiments'] = fichier_4

# 5. Statistiques par classe DPE
fichier_5 = os.path.join(chemin_sortie, 'stats_par_classe_dpe.csv')
stats_dpe.to_csv(fichier_5, index=False, sep=';', encoding='utf-8-sig')
exports['Statistiques par classe DPE'] = fichier_5

# 6. Comparaison theorique vs reelle
fichier_6 = os.path.join(chemin_sortie, 'comparaison_dpe_theorique_vs_reel.csv')
comparaison_dpe.to_csv(fichier_6, index=False, sep=';', encoding='utf-8-sig')
exports['Comparaison DPE theorique vs reel'] = fichier_6

# 7. Evolution mensuelle
fichier_7 = os.path.join(chemin_sortie, 'evolution_mensuelle_par_energie.csv')
conso_mensuelle.to_csv(fichier_7, sep=';', encoding='utf-8-sig')
exports['Evolution mensuelle'] = fichier_7

print("=" * 80)
print("EXPORT DES TABLEAUX DE SYNTHESE")
print("=" * 80)
print()
print(f"Repertoire de sortie : {os.path.abspath(chemin_sortie)}")
print()
for nom, chemin in exports.items():
    taille = os.path.getsize(chemin)
    print(f"  -> {nom} : {os.path.basename(chemin)} ({taille:,} octets)")

print(f"\nTotal : {len(exports)} fichiers exportes.")

## Conclusion

### Principaux enseignements de l'analyse descriptive

**1. Par type d'energie :**
- L'electricite et le gaz representent les postes de consommation les plus importants.
- La variabilite (coefficient de variation) permet d'identifier les energies avec les plus grandes disparites entre batiments.

**2. Par type de batiment :**
- Les piscines et gymnases sont generalement les batiments les plus energivores en raison de leurs besoins specifiques (chauffage de l'eau, ventilation).
- Les mediatheques et mairies presentent des consommations plus moderees.

**3. Par commune :**
- Les ecarts entre communes s'expliquent par le nombre et le type de batiments publics presents.
- Le tri par cout total permet d'identifier les communes ou les efforts de renovation seraient les plus rentables.

**4. Batiments extremes :**
- L'identification des batiments les plus et moins energivores fournit des pistes concretes pour les actions de renovation energetique.
- Le ratio entre le plus et le moins energivore revele l'ampleur des disparites.

**5. Classes DPE :**
- L'analyse permet de verifier si la classification DPE est coherente avec les consommations reelles mesurees.
- Les ecarts entre theorique et reel revelent les limites du diagnostic energetique ponctuel.

**6. Evolution temporelle :**
- La comparaison 2023/2024 permet de detecter des tendances d'amelioration ou de degradation.
- La saisonnalite confirme les pics de consommation en hiver pour le chauffage.

**7. Theorique vs reel :**
- La comparaison avec les seuils DPE theoriques permet d'identifier les batiments qui surconsomment ou sous-consomment par rapport a leur classe.
- Ces ecarts constituent des indicateurs d'alerte pour la maintenance ou la renovation.

---
*Tableaux de synthese exportes dans `../output/` au format CSV pour exploitation ulterieure.*